# Prueba de BERT

## Descarga e instalación de recursos

Para realizar las pruebas sobre BERT vamos a necesitar una serie de recursos que nos permitan acceder a su funcionalidad.

La biblioteca que utilizaremos para acceder a ellos será la biblioteca Transformers de HuggingFace. Este módulo contiene una serie de funcionalidades que permiten realizar funciones de procesamiento de lenguaje, en nuestro caso concreto, generación de lenguaje natural.

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 43.0 MB/s 
     |████████████████████████████████| 880 kB 4.0 MB/s 
     |████████████████████████████████| 77 kB 2.7 MB/s 
     |████████████████████████████████| 596 kB 47.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=22fe89b5b0b0f604a8528d0f236f417c515de17a998fbcce59502c2384d9640b
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Bibliotecas necesarias

A continuación vamos a importar los módulos necesarios para probar el funcionamiento de bert.

En nuestro caso utilizaremos, el módulo *BertForMaskedLM* que implementa el modelo de lenguaje BERT para lenguaje enmascrado y *BertTokenizer* utilizado para el proceso de tokenización.

In [4]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch

## Tokenizer

El primer paso para poder probar el modelo es convertir los datos de entrada a una secuencia de identificdores de tokens o *tokens ids* comprensibles por el modelo de lenguaje. Utilizaremos la version 'bert-case-uncased' del tokenizador *BertTokenizer* para realizar este proceso.

In [105]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

En la siguiente oración, se puede comprobar como separa el tokenizador cada una de las palabras atendiendo a los subtokens de los que están compuestas. De esta manera, 'thunderous' se compone de 'thunder' y '##ous'

In [106]:
sequence = "The thunderous roar of the jet overhead confirmed her worst fears"
tokenized_sequence = tokenizer.tokenize(sequence)
print(tokenized_sequence)

['the', 'thunder', '##ous', 'roar', 'of', 'the', 'jet', 'overhead', 'confirmed', 'her', 'worst', 'fears']


A continuación, comprobamos la salida de este proceso de tokenización. Los resultados devueltos por este procedimiento son los *input_ids* correspondientes de la secuencia, *token_type_ids* y *attention_mask* o máscara de atención.

In [107]:
tokenizer('Hello world?')

{'input_ids': [101, 7592, 2088, 1029, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [108]:
sequence = "Hello world?"
tokenized_sequence = tokenizer.tokenize(sequence)
print(tokenized_sequence)

['hello', 'world', '?']


## Carga del modelo

Primero descargamos el modelo concreto que vamos a utilizar. En nuestro caso, emplearemos la versión 'bert-base-uncased' del modelo.

In [110]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased',return_dict = True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Resultados

A continuación, se comprobará un ejemplo concreto. En este caso estamos utilizando BERT para generación de lenguaje enmascarado, así que dada una oración, ocultaremos alguna de sus palabras bajo el token de máscara del tokenizador y le pasaremos al modelo esta entrada tokenizada.

In [111]:
tokenizer.mask_token

'[MASK]'

In [112]:
tokenizer.mask_token_id

103

In [113]:
text = "Every Monday, Mary goes to the " + tokenizer.mask_token + " to relax."
input = tokenizer.encode_plus(text, return_tensors = "pt")
# indice del token [MASK]
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
print(f'index:  {int(mask_index[0][0])}')

index:  8


In [114]:
tokenizer.decode(input['input_ids'][0])

'[CLS] every monday, mary goes to the [MASK] to relax. [SEP]'

In [115]:
input

{'input_ids': tensor([[ 101, 2296, 6928, 1010, 2984, 3632, 2000, 1996,  103, 2000, 9483, 1012,
          102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Para finalizar, generamos el texto a partir del modelo.

In [116]:
output = model(**input)

In [117]:
logits = output.logits
print(logits)
softmax = F.softmax(logits, dim = -1)
print(softmax)
mask_word = softmax[0, mask_index, :]
print(mask_word)
top_5 = torch.topk(mask_word, 5, dim = 1)[1][0]
for token in top_5:
   word = tokenizer.decode([token])
   new_sentence = text.replace(tokenizer.mask_token, word)
   print(new_sentence)

tensor([[[ -6.7618,  -6.7374,  -6.7210,  ...,  -6.0865,  -5.9108,  -4.0268],
         [ -8.7497,  -8.9794,  -9.1654,  ...,  -8.2566,  -6.8753,  -6.5967],
         [ -7.8636,  -8.0510,  -7.8461,  ...,  -6.3059,  -5.4562,  -7.7335],
         ...,
         [ -9.0204,  -9.4107,  -9.2717,  ...,  -8.7517,  -8.8832,  -5.2407],
         [-10.6601, -10.3545, -10.6067,  ...,  -8.4040,  -9.4216,  -6.8385],
         [-14.5783, -14.8022, -14.7373,  ..., -14.2305, -11.7988,  -8.1406]]],
       grad_fn=<AddBackward0>)
tensor([[[4.5826e-07, 4.6954e-07, 4.7731e-07,  ..., 9.0031e-07,
          1.0731e-06, 7.0611e-06],
         [9.3756e-14, 7.4516e-14, 6.1865e-14,  ..., 1.5352e-13,
          6.1099e-13, 8.0731e-13],
         [2.0734e-12, 1.7190e-12, 2.1099e-12,  ..., 9.8439e-12,
          2.3024e-11, 2.3614e-12],
         ...,
         [3.2964e-12, 2.2312e-12, 2.5640e-12,  ..., 4.3124e-12,
          3.7814e-12, 1.4439e-10],
         [9.2773e-16, 1.2593e-15, 9.7859e-16,  ..., 8.8554e-15,
          3.2010e

In [118]:
logits = output.logits
softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
top_5 = torch.topk(mask_word, 5, dim = 1)[1][0]
for token in top_5:
   word = tokenizer.decode([token])
   new_sentence = text.replace(tokenizer.mask_token, word)
   print(new_sentence)

Every Monday, Mary goes to the beach to relax.
Every Monday, Mary goes to the spa to relax.
Every Monday, Mary goes to the hospital to relax.
Every Monday, Mary goes to the gym to relax.
Every Monday, Mary goes to the pool to relax.


En los resultados anteriores podemos comprobar las 5 palabras que mayor probabilidad tienen de encajar en esta palabra oculta de la secuencia de entrada. En este orden, serían: 

  * beach
  * spa
  * hospital
  * gym
  * pool